In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob # For finding result files
from IPython.display import display, HTML

# --- Configuration ---
# Define the path to your results directory (where the dated subfolder is)
# This should match 'results_dir_timedated' from your experiment script
BASE_RESULTS_DIR = r"G:\My Drive\code_projects\btc_forecast\results" # <<< UPDATE THIS TO YOUR LOCAL PATH
FREQUENCY = "15m" # <<< Ensure this matches the frequency of results you want to analyze
DATE_TODAY_RESULTS = "20250507" # <<< UPDATE THIS to the date folder of your results

RESULTS_PATH = os.path.join(BASE_RESULTS_DIR, f"{FREQUENCY}_{DATE_TODAY_RESULTS}")

# Number of initial OOS chunks to average for "Initial OOS Score"
NUM_INITIAL_OOS_CHUNKS_FOR_AVG = 3

print(f"Results Path: {RESULTS_PATH}")
pd.set_option('display.float_format', '{:.6f}'.format)

Results Path: G:\My Drive\code_projects\btc_forecast\results\15m_20250507


In [2]:
json_pattern = os.path.join(RESULTS_PATH, f"*_{FREQUENCY}_*_healthspan_summary.json")
result_files = glob.glob(json_pattern)

all_results_data = []
if not result_files:
    print(f"No result files found matching pattern: {json_pattern}")
else:
    print(f"Found {len(result_files)} result files to load.")
    for file_path in result_files:
        try:
            with open(file_path, 'r') as f:
                data = json.load(f)
                # Each file contains a list of results (one per candle_count for a specific asset/target)
                all_results_data.extend(data)
        except Exception as e:
            print(f"Error loading or parsing {file_path}: {e}")

if not all_results_data:
    print("No data loaded. Please check RESULTS_PATH and file contents.")
    # Stop execution or handle error
    raise ValueError("No results data loaded.")
else:
    all_results_df = pd.DataFrame(all_results_data)
    print(f"\nSuccessfully loaded {len(all_results_df)} records into a DataFrame.")
    display(all_results_df.head())
    print("\nDataFrame Info:")
    all_results_df.info()

Found 2556 result files to load.

Successfully loaded 25420 records into a DataFrame.


,target,asset,candle_count,best_score_val_internal,error,time_taken_seconds,num_data_points_train,num_val_windows_used,external_eval_wql_scores,healthspan_steps,wql_threshold
0,volume,1000CATS_USDT_USDT,100,-0.036143,None,3.217910,100,10,"[{'time_since_train': 90000.0, 'WQL': -0.02215...",1000,0.032179
1,volume,1000CATS_USDT_USDT,250,-0.027089,None,4.562007,250,10,"[{'time_since_train': 90000.0, 'WQL': -0.02215...",1000,0.041234
2,volume,1000CATS_USDT_USDT,500,-0.027193,None,6.333754,500,10,"[{'time_since_train': 90000.0, 'WQL': -0.02215...",1000,0.041130
3,volume,1000CATS_USDT_USDT,750,-0.027276,None,9.387293,750,10,"[{'time_since_train': 90000.0, 'WQL': -0.02215...",1000,0.041047
4,volume,1000CATS_USDT_USDT,1000,-0.027430,None,11.491380,1000,10,"[{'time_since_train': 90000.0, 'WQL': -0.02215...",1000,0.040892



DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25420 entries, 0 to 25419
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   target                    25420 non-null  object 
 1   asset                     25420 non-null  object 
 2   candle_count              25420 non-null  int64  
 3   best_score_val_internal   25420 non-null  float64
 4   error                     0 non-null      object 
 5   time_taken_seconds        25420 non-null  float64
 6   num_data_points_train     25420 non-null  int64  
 7   num_val_windows_used      25420 non-null  int64  
 8   external_eval_wql_scores  25420 non-null  object 
 9   healthspan_steps          25420 non-null  int64  
 10  wql_threshold             25360 non-null  float64
dtypes: float64(3), int64(4), object(4)
memory usage: 2.1+ MB


In [3]:
def process_results(df):
    processed_records = []
    for _, row in df.iterrows():
        record = row.to_dict()

        # 1. Convert internal score (-WQL) to actual IS WQL (lower is better)
        if pd.notna(record.get("best_score_val_internal")):
            record['actual_is_wql'] = -record['best_score_val_internal']
        else:
            record['actual_is_wql'] = np.nan

        # 2. Process external_eval_wql_scores
        oos_wql_scores_finite = [] # Store only finite scores
        if isinstance(record.get("external_eval_wql_scores"), list):
            for score_entry in record["external_eval_wql_scores"]:
                if isinstance(score_entry, dict):
                    wql_val = score_entry.get("WQL")
                    # Ensure the value is a number and is finite (not NaN, inf, -inf)
                    if isinstance(wql_val, (int, float, np.number)) and np.isfinite(wql_val):
                        oos_wql_scores_finite.append(float(wql_val))

        if oos_wql_scores_finite: # Check if the list of finite scores is not empty
            # 3. Initial OOS WQL Mean (from first N finite chunks)
            record['initial_oos_wql_mean'] = np.mean(oos_wql_scores_finite[:NUM_INITIAL_OOS_CHUNKS_FOR_AVG]) # np.mean is fine now

            # 4. OOS WQL Standard Deviation (stability)
            if len(oos_wql_scores_finite) >= 2: # Std dev needs at least 2 points
                record['oos_wql_std'] = np.std(oos_wql_scores_finite) # np.std is fine now
            elif len(oos_wql_scores_finite) == 1:
                record['oos_wql_std'] = 0.0 # Std dev of a single point is 0
            else: # Should not happen if oos_wql_scores_finite is not empty, but as safeguard
                record['oos_wql_std'] = np.nan
        else:
            record['initial_oos_wql_mean'] = np.nan
            record['oos_wql_std'] = np.nan

        # 5. IS-OOS Gap
        if pd.notna(record['actual_is_wql']) and pd.notna(record['initial_oos_wql_mean']):
            record['is_oos_gap'] = record['initial_oos_wql_mean'] - record['actual_is_wql']
        else:
            record['is_oos_gap'] = np.nan

        processed_records.append(record)

    return pd.DataFrame(processed_records)

if 'all_results_df' in locals() and not all_results_df.empty:
    processed_df = process_results(all_results_df.copy()) # Apply the updated function
    print("Processed DataFrame with new metrics:")
    display_cols_processed = [
        'asset', 'target', 'candle_count',
        'actual_is_wql', 'initial_oos_wql_mean', 'is_oos_gap', 'oos_wql_std',
        'healthspan_steps', 'wql_threshold', 'error'
    ]
    # Ensure columns exist before trying to display them
    display_cols_present = [col for col in display_cols_processed if col in processed_df.columns]
    if display_cols_present:
        display(processed_df[display_cols_present].head())
    else:
        display(processed_df.head()) # Fallback to display all if key columns are missing
else:
    print("Skipping processing as 'all_results_df' is not available.")

Processed DataFrame with new metrics:


,asset,target,candle_count,actual_is_wql,initial_oos_wql_mean,is_oos_gap,oos_wql_std,healthspan_steps,wql_threshold,error
0,1000CATS_USDT_USDT,volume,100,0.036143,-0.071130,-0.107273,0.045548,1000,0.032179,None
1,1000CATS_USDT_USDT,volume,250,0.027089,-0.071130,-0.098218,0.045548,1000,0.041234,None
2,1000CATS_USDT_USDT,volume,500,0.027193,-0.071130,-0.098322,0.045548,1000,0.041130,None
3,1000CATS_USDT_USDT,volume,750,0.027276,-0.071130,-0.098405,0.045548,1000,0.041047,None
4,1000CATS_USDT_USDT,volume,1000,0.027430,-0.071130,-0.098560,0.045548,1000,0.040892,None


In [4]:
# @title Analysis 3: Overall Best Effort Candle Count & Associated Healthspan

if 'processed_df' in locals() and not processed_df.empty:
    print("\n--- Analysis 3: Overall Best Effort Candle Count by Combined Criteria ---")

    overall_summary_list = []
    # Group by asset and target, as requested
    grouped_for_best_effort = processed_df.groupby(['asset', 'target'])

    for (asset, target), group_df_orig in grouped_for_best_effort:
        if group_df_orig.empty:
            print(f"Skipping {asset} - {target}: Empty group.")
            continue

        group_df = group_df_orig.copy() # Work on a copy to avoid SettingWithCopyWarning

        # Filter for rows where all necessary metrics for ranking are available
        metrics_for_ranking = ['initial_oos_wql_mean', 'is_oos_gap', 'oos_wql_std']
        # Also need these for reporting
        reporting_metrics = ['candle_count', 'healthspan_steps', 'actual_is_wql', 'wql_threshold']
        
        # Keep only rows where all ranking metrics are non-NaN
        group_df.dropna(subset=metrics_for_ranking, inplace=True)

        if group_df.empty:
            print(f"[{asset}/{target}] Skipping: Not enough valid data after NaN drop for ranking metrics.")
            overall_summary_list.append({
                'asset': asset, 'target': target,
                'Best_Effort_CC': np.nan, 'Initial_OOS_WQL': np.nan,
                'IS_OOS_Gap': np.nan, 'OOS_WQL_Std': np.nan,
                'Healthspan_at_Best_Effort_CC': np.nan,
                'Actual_IS_WQL_at_Best_Effort_CC': np.nan,
                'WQL_Threshold_at_Best_Effort_CC': np.nan,
                'Comment': 'Insufficient data for ranking'
            })
            continue

        # Calculate ranks for each criterion (lower value of metric = better rank)
        # 'min' method assigns the same rank to tied values
        group_df['rank_initial_oos'] = group_df['initial_oos_wql_mean'].rank(method='min', na_option='bottom')
        group_df['abs_is_oos_gap'] = group_df['is_oos_gap'].abs() # We rank based on absolute gap
        group_df['rank_is_oos_gap'] = group_df['abs_is_oos_gap'].rank(method='min', na_option='bottom')
        group_df['rank_oos_std'] = group_df['oos_wql_std'].rank(method='min', na_option='bottom')

        # Composite score (sum of ranks - lower composite rank is better)
        group_df['composite_rank'] = group_df['rank_initial_oos'] + \
                                     group_df['rank_is_oos_gap'] + \
                                     group_df['rank_oos_std']

        # Find the row with the best composite rank
        # In case of ties in composite_rank, further sort by:
        # 1. initial_oos_wql_mean (lower is better)
        # 2. oos_wql_std (lower is better)
        # 3. is_oos_gap (absolute, lower is better)
        # 4. candle_count (prefer smaller candle_count in case of further ties, optional)
        best_overall_row = group_df.sort_values(
            by=['composite_rank', 'initial_oos_wql_mean', 'oos_wql_std', 'abs_is_oos_gap', 'candle_count']
        ).iloc[0]

        overall_summary_list.append({
            'asset': asset,
            'target': target,
            'Best_Effort_CC': int(best_overall_row['candle_count']),
            'Initial_OOS_WQL': best_overall_row['initial_oos_wql_mean'],
            'IS_OOS_Gap': best_overall_row['is_oos_gap'], # Report original gap
            'OOS_WQL_Std': best_overall_row['oos_wql_std'],
            'Healthspan_at_Best_Effort_CC': best_overall_row['healthspan_steps'],
            'Actual_IS_WQL_at_Best_Effort_CC': best_overall_row['actual_is_wql'],
            'WQL_Threshold_at_Best_Effort_CC': best_overall_row['wql_threshold'],
            'Composite_Rank': best_overall_row['composite_rank'], # For reference/debugging
            'Comment': '' # For any notes
        })

    if overall_summary_list:
        final_overall_summary_df = pd.DataFrame(overall_summary_list)
        print("\n--- Overall 'Best Effort' Candle Count Summary Table ---")
        print("This table shows one 'best effort' candle_count per asset/target, chosen by ranking performance across multiple criteria.")
        
        # Select and order columns for the final display
        display_summary_cols = [
            'asset', 'target', 'Best_Effort_CC',
            'Actual_IS_WQL_at_Best_Effort_CC', # Added for context
            'Initial_OOS_WQL', 'IS_OOS_Gap', 'OOS_WQL_Std',
            'Healthspan_at_Best_Effort_CC',
            'WQL_Threshold_at_Best_Effort_CC', # Added for context
            'Comment'
        ]
        # Ensure all selected columns exist to prevent KeyError
        display_summary_cols = [col for col in display_summary_cols if col in final_overall_summary_df.columns]

        display(final_overall_summary_df[display_summary_cols].head())  # (HTML(final_overall_summary_df[display_summary_cols].to_html(na_rep="-")))
    else:
        print("Could not generate the overall best effort summary table. No valid data after grouping or ranking.")
else:
    print("Skipping Analysis 3 (Overall Best Effort) as 'processed_df' is not available.")


--- Analysis 3: Overall Best Effort Candle Count by Combined Criteria ---

--- Overall 'Best Effort' Candle Count Summary Table ---
This table shows one 'best effort' candle_count per asset/target, chosen by ranking performance across multiple criteria.


,asset,target,Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Comment
0,1000000BABYDOGE_USDT_USDT,close,100,0.001689,-0.001577,-0.003267,0.000922,1000,-0.000306,
1,1000000BABYDOGE_USDT_USDT,high,100,0.000962,-0.000794,-0.001757,0.000765,1000,0.000185,
2,1000000BABYDOGE_USDT_USDT,low,100,0.001082,-0.001434,-0.002516,0.000567,1000,-0.000231,
3,1000000BABYDOGE_USDT_USDT,open,1000,0.001608,-0.001520,-0.003128,0.000732,1000,-0.000511,
4,1000000BABYDOGE_USDT_USDT,volume,500,0.019030,-0.031458,-0.050487,0.010892,1000,-0.002691,


In [5]:
final_overall_summary_df.describe()

C:\Users\gjnet\code_projects\btc_forecast\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\gjnet\code_projects\btc_forecast\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\Users\gjnet\code_projects\btc_forecast\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Composite_Rank
count,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2555.000000,2549.000000,2555.000000
mean,513.679061,-0.010805,-inf,0.009716,765.166341,inf,-inf,3.098630
std,352.043713,0.024530,NaN,0.105231,383.212631,NaN,NaN,0.716498
min,100.000000,-0.406870,-inf,0.000000,0.000000,0.000001,-inf,3.000000
25%,100.000000,-0.005334,-0.009699,0.000653,500.000000,0.001258,-0.000862,3.000000
50%,500.000000,-0.002068,-0.004026,0.001162,1000.000000,0.001855,-0.000203,3.000000
75%,750.000000,-0.001329,-0.002761,0.004020,1000.000000,0.004309,0.000941,3.000000
max,1000.000000,-0.000005,-0.000007,3.934376,1000.000000,inf,5.204923,11.000000


In [6]:
final_overall_summary_df["Healthspan_at_Best_Effort_CC"].value_counts()

Healthspan_at_Best_Effort_CC
1000    1775
0        305
100      113
200       70
300       64
600       60
400       53
500       45
700       24
800       24
900       22
Name: count, dtype: int64

In [7]:
vc = final_overall_summary_df["Best_Effort_CC"].value_counts()
display(vc)

Best_Effort_CC
100     777
750     628
1000    535
250     315
500     300
Name: count, dtype: int64

In [8]:
asset_selection_df = final_overall_summary_df[final_overall_summary_df["Best_Effort_CC"] == vc.index[0]]
asset_selection_df = asset_selection_df[asset_selection_df["Healthspan_at_Best_Effort_CC"] > 0]
hs_vc = asset_selection_df["Healthspan_at_Best_Effort_CC"].value_counts()
display(hs_vc)

Healthspan_at_Best_Effort_CC
1000    580
100      31
400      23
200      18
600      16
300      14
500      13
800       9
700       7
900       6
Name: count, dtype: int64

In [9]:
filtered_asset_sel_df = asset_selection_df[asset_selection_df["Healthspan_at_Best_Effort_CC"] == hs_vc.index[0]]
display(filtered_asset_sel_df.head())
selected_assets = filtered_asset_sel_df["asset"].drop_duplicates().tolist()

,asset,target,Best_Effort_CC,Initial_OOS_WQL,IS_OOS_Gap,OOS_WQL_Std,Healthspan_at_Best_Effort_CC,Actual_IS_WQL_at_Best_Effort_CC,WQL_Threshold_at_Best_Effort_CC,Composite_Rank,Comment
0,1000000BABYDOGE_USDT_USDT,close,100,-0.001577,-0.003267,0.000922,1000,0.001689,-0.000306,3.000000,
1,1000000BABYDOGE_USDT_USDT,high,100,-0.000794,-0.001757,0.000765,1000,0.000962,0.000185,3.000000,
2,1000000BABYDOGE_USDT_USDT,low,100,-0.001434,-0.002516,0.000567,1000,0.001082,-0.000231,3.000000,
6,1000000CHEEMS_USDT_USDT,high,100,-0.001483,-0.002924,0.002217,1000,0.001441,0.001885,3.000000,
9,1000000CHEEMS_USDT_USDT,volume,100,-0.101828,-0.137120,0.064936,1000,0.035292,0.062112,3.000000,


In [10]:
# @title Display All Processed Metrics for Specific Candle Counts

# <<< --- DEFINE YOUR CANDLE COUNTS OF INTEREST HERE --- >>>
# For example, if your experiment ran with [100, 250, 500, 750, 1000, 1500, 2000]
# and now you only want to see detailed results for [500, 1000]
specific_candle_counts_to_view = [100, 250, 500] # <<< MODIFY THIS LIST WITH YOUR CHOICES

print(f"Filtering results to show only for candle_counts: {specific_candle_counts_to_view}")

if 'processed_df' in locals() and not processed_df.empty:
    # Filter the DataFrame for the specified candle_counts
    filtered_results_df = processed_df[processed_df['candle_count'].isin(specific_candle_counts_to_view)][display_cols_processed].copy()

    if filtered_results_df.empty:
        print(f"No results found in the loaded data for the specified candle counts: {specific_candle_counts_to_view}")
    else:
        print(f"\nDisplaying all processed columns for selected candle_counts: {specific_candle_counts_to_view}")
        
        # Ensure all columns are displayed by pandas
        # pd.set_option('display.max_columns', None)
        # pd.set_option('display.max_rows', 200) # Show more rows if needed
        # pd.set_option('display.width', 1000)   # Adjust terminal width if needed

        # Sort for easier comparison (optional, but recommended)
        # This will group by asset, then target, then by the selected candle_count
        filtered_results_df_sorted = filtered_results_df.sort_values(
            by=['asset', 'target', 'candle_count']
        ).reset_index(drop=True)
        
        # Display the filtered DataFrame with all its columns
        # Using HTML for better rendering in Jupyter
        display(filtered_results_df_sorted.head())  # (HTML(filtered_results_df_sorted.to_html(na_rep="-")))
        
        print(f"\nTable above shows all {len(filtered_results_df_sorted.columns)} processed columns for your selected candle counts.")
        print("Relevant columns for your questions include:")
        print("- 'asset', 'target', 'candle_count'")
        print("- 'actual_is_wql' (In-Sample WQL, lower is better)")
        print("- 'initial_oos_wql_mean' (Avg WQL of first few Out-of-Sample chunks, lower is better)")
        print("- 'is_oos_gap' (Difference: Initial OOS WQL - IS WQL, smaller absolute value is better)")
        print("- 'oos_wql_std' (Stability/consistency of OOS WQL, lower is better)")
        print("- 'healthspan_steps' (How long model stayed good, higher is better)")
        print("- 'wql_threshold' (The actual WQL error limit for healthspan)")
        print("- 'best_score_val_internal' (Original -WQL from AutoGluon internal validation, higher is better)")
        print("- 'error' (Any errors during processing)")

else:
    print("Cannot filter results as 'processed_df' is not available. Please ensure previous cells, especially data loading and processing, have run successfully.")

Filtering results to show only for candle_counts: [100, 250, 500]

Displaying all processed columns for selected candle_counts: [100, 250, 500]


,asset,target,candle_count,actual_is_wql,initial_oos_wql_mean,is_oos_gap,oos_wql_std,healthspan_steps,wql_threshold,error
0,1000000BABYDOGE_USDT_USDT,close,100,0.001689,-0.001577,-0.003267,0.000922,1000,-0.000306,None
1,1000000BABYDOGE_USDT_USDT,close,100,0.001689,-0.001577,-0.003267,0.000922,1000,-0.000306,None
2,1000000BABYDOGE_USDT_USDT,close,250,0.001867,-0.001577,-0.003444,0.000922,1000,-0.000484,None
3,1000000BABYDOGE_USDT_USDT,close,250,0.001867,-0.001577,-0.003444,0.000922,1000,-0.000484,None
4,1000000BABYDOGE_USDT_USDT,close,500,0.001768,-0.001577,-0.003345,0.000922,1000,-0.000385,None



Table above shows all 10 processed columns for your selected candle counts.
Relevant columns for your questions include:
- 'asset', 'target', 'candle_count'
- 'actual_is_wql' (In-Sample WQL, lower is better)
- 'initial_oos_wql_mean' (Avg WQL of first few Out-of-Sample chunks, lower is better)
- 'is_oos_gap' (Difference: Initial OOS WQL - IS WQL, smaller absolute value is better)
- 'oos_wql_std' (Stability/consistency of OOS WQL, lower is better)
- 'healthspan_steps' (How long model stayed good, higher is better)
- 'wql_threshold' (The actual WQL error limit for healthspan)
- 'best_score_val_internal' (Original -WQL from AutoGluon internal validation, higher is better)
- 'error' (Any errors during processing)
